In [ ]:
from binance.client import Client 
from backtesting import Strategy, Backtest
import pandas as pd 
import importlib
    
from app.strategy.macd_strategy_bt import MACDStrategyBacktest

import webbrowser
import os

/Users/Lynn/Documents/coding_workspace/QF635_Market_Microstructure/.venv/lib/python3.10/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

## Original MACD Strategy

In [2]:
client = Client(api_key='', api_secret='')
klines = client.get_historical_klines(
    symbol='BTCUSDT',
    interval=Client.KLINE_INTERVAL_1HOUR,
    start_str='2024-01-01',
    end_str='2024-06-30'
)

In [3]:
df_klines = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume',
    'close_time', 'quote_asset_volume', 'num_trades',
    'taker_buy_base', 'taker_buy_quote', 'ignore'])
df_klines

,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,num_trades,taker_buy_base,taker_buy_quote,ignore
0,1704067200000,42283.58000000,42554.57000000,42261.02000000,42475.23000000,1271.68108000,1704070799999,53957248.97378900,47134,682.57581000,28957416.81964500,0
1,1704070800000,42475.23000000,42775.00000000,42431.65000000,42613.56000000,1196.37856000,1704074399999,50984893.34814160,50396,712.32227000,30355645.34827640,0
2,1704074400000,42613.57000000,42638.41000000,42500.00000000,42581.10000000,685.21980000,1704077999999,29167377.97323750,29863,288.98864000,12301017.29976200,0
3,1704078000000,42581.09000000,42586.64000000,42230.08000000,42330.49000000,794.80391000,1704081599999,33709050.27411360,38620,356.37209000,15112996.37584990,0
4,1704081600000,42330.50000000,42399.99000000,42209.46000000,42399.99000000,715.41760000,1704085199999,30271621.00073900,36038,371.12012000,15703618.93198530,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4340,1719691200000,61057.38000000,61088.40000000,60970.00000000,61050.00000000,319.46435000,1719694799999,19499979.32515800,23096,142.84685000,8718800.36949500,0
4341,1719694800000,61050.00000000,61090.00000000,60944.94000000,61044.99000000,319.53467000,1719698399999,19498161.02604430,23221,138.72008000,8465072.43230030,0
4342,1719698400000,61044.99000000,61060.00000000,60839.51000000,60970.00000000,362.76005000,1719701999999,22115851.69725870,35816,144.60976000,8814765.75794620,0
4343,1719702000000,60970.00000000,61059.37000000,60940.00000000,60986.68000000,202.05321000,1719705599999,12325609.80243060,24516,88.08706000,5373443.31435060,0


In [4]:
df_klines['timestamp'] = pd.to_datetime(df_klines['timestamp'], unit='ms')
df_klines.set_index('timestamp', inplace=True)
df_klines = df_klines[['open', 'high', 'low', 'close', 'volume']].astype(float)
df_klines.columns = ['Open', 'High', 'Low', 'Close', 'Volume']

In [ ]:
bt = Backtest(df_klines, MACDStrategyBacktest, cash=100000, commission=0.0004)
stats = bt.run()
bt.plot(filename='macd_backtest_plot', open_browser=False)

GridPlot(id='p1398', ...)

In [6]:
print(stats)

Start                     2024-01-01 00:00:00
End                       2024-06-30 00:00:00
Duration                    181 days 00:00:00
Exposure Time [%]                    52.52014
Equity Final [$]                 110653.50635
Equity Peak [$]                  121226.94554
Commissions [$]                    9646.63723
Return [%]                           10.65351
Buy & Hold Return [%]                43.67091
Return (Ann.) [%]                    16.38383
Volatility (Ann.) [%]                29.52623
CAGR [%]                              22.6476
Sharpe Ratio                          0.55489
Sortino Ratio                         1.18908
Calmar Ratio                          1.21024
Alpha [%]                            -2.74945
Beta                                  0.30691
Max. Drawdown [%]                   -13.53762
Avg. Drawdown [%]                    -2.02413
Max. Drawdown Duration      108 days 16:00:00
Avg. Drawdown Duration        6 days 23:00:00
# Trades                          

In [7]:
print("==== Portfolio Summary ====")
print("Sharpe Ratio:", stats['Sharpe Ratio'])
print("Total PnL:        ", stats['Equity Final [$]'] - float(100000))
print("Realized PnL:     ", stats['_trades']['PnL'].sum())
print("Total Commission: ", stats['Commissions [$]'])
print("Unrealized PnL:   ", stats['_equity_curve']['Equity'].iloc[-1] - stats['_equity_curve']['Equity'].iloc[-2])

==== Portfolio Summary ====
Sharpe Ratio: 0.5548908808383813
Total PnL:         10653.50635200004
Realized PnL:      20065.950000000004
Total Commission:  9646.637227999998
Unrealized PnL:    37.870000000009895


In [ ]:
file_path = os.path.abspath("macd_backtest_plot.html")
webbrowser.open(f"file://{file_path}")

True

## Dual MACD Strategy